In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
# -*- coding: utf-8 -*-
from __future__ import division
import numpy as np
import tensorflow as tf

class TFNMF(object):
    """Non-negative Matrix Factorization by TensorFlow"""
    def __init__(self, V, rank):

        #Numpy行列からTensorFlowのTensorに変換
        V_ = tf.constant(V, dtype=tf.float32)
        shape = V.shape

        #平均 sqrt(V.mean() / rank) の一様乱数にスケール
        scale = 2 * np.sqrt(V.mean() / rank)
        initializer = tf.random_uniform_initializer(maxval=scale)

        #行列H,Wの変数生成
        self.H = H = tf.get_variable("H", [rank, shape[1]],
                                     initializer=initializer)
        self.W = W = tf.get_variable(name="W", shape=[shape[0], rank],
                                     initializer=initializer)

        #収束判定のためにWを保存する
        W_old = tf.get_variable(name="W_old", shape=[shape[0], rank])
        self._save_W = W_old.assign(W)

        #MUアルゴリズム
        #Hを更新する
        Wt = tf.transpose(W)
        WV = tf.matmul(Wt, V_)
        WWH = tf.matmul(tf.matmul(Wt, W), H)
        WV_WWH = WV / WWH
#         with tf.device('/cpu:0'):
#             #0割りでnanが入った要素を0に変換する
#             WV_WWH = tf.select(tf.is_nan(WV_WWH),
#                               tf.zeros_like(WV_WWH),
#                               WV_WWH)
        H_new = H * WV_WWH
        self._update_H = H.assign(H_new)

        #Wを更新する(Hは更新済み)
        Ht = tf.transpose(H)
        VH = tf.matmul(V_, Ht)
        WHH = tf.matmul(W, tf.matmul(H, Ht))
        VH_WHH = VH / WHH
#         with tf.device('/cpu:0'):
#             #0割りでnanが入った要素を0に変換する
#             WV_WWH = tf.select(tf.is_nan(WV_WWH),
#                               tf.zeros_like(WV_WWH),
#                               WV_WWH)
        W_new = W * VH_WHH
        self._update_W = W.assign(W_new)

        #Wの各要素の変化総量
        self._delta = tf.reduce_sum(tf.abs(W_old - W))

    def run(self, sess, max_iter=100):
        tf.initialize_all_variables().run()
        for i in range(max_iter):
            sess.run(self._save_W)
            H, _ = sess.run([self.H, self._update_H])
            W, _ = sess.run([self.W, self._update_W])
            delta = sess.run(self._delta)
            if delta < 0.001:
                break
        return W, H

In [3]:
import time
V = np.random.rand(10000,10000)
rank = 1000
num_core = 8

tfnmf = TFNMF(V, rank)
config = tf.ConfigProto(inter_op_parallelism_threads=num_core,
                       intra_op_parallelism_threads=num_core)
with tf.Session(config=config) as sess:
    start = time.time()
    W, H = tfnmf.run(sess)
    print("Computational Time: ", time.time() - start)

#２乗誤差計算
W = np.mat(W)
H = np.mat(H)
error = np.power(V - W * H, 2).sum()
print("Reconstruction Error: ", error)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Computational Time:  612.3173954486847
Reconstruction Error:  8182436.35589
